# project 4
---
## Predicting The 2024 Presidential Election outcome with Machine Learning algorithm
---
### Retrieving the dataset

In [ ]:
# Import findspark and initialize. 
# import findspark
# findspark.init()

# Import packages
# from pyspark.sql import SparkSession
# import time

# Create a SparkSession
#spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
# 1. Read in the AWS S3 bucket into a DataFrame.
#from pyspark import SparkFiles
#url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
#spark.sparkContext.addFile(url)
# df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
# df.show()

In [5]:
# Import our dependencies
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path

#  Import and read the dataset
df = pd.read_csv(" ")
df.head()


In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

### Preprocessing
---

#### Data cleaning and normalization

In [ ]:
# Determine the number of unique values in each column.
df.nunique()

In [7]:
# Drop the non-beneficial columns

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`

In [ ]:
# Split our preprocessed data into our features and target arrays
y = 
X = 

# Split the preprocessed data into a training and testing dataset
X_train , X_test , y_train , y_test = train_test_split(X,y , random_state = 78)

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [ ]:

# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver = 'lbfgs' , random_state = 1)
classifier

# Fit the model using training data
classifier.fit(X_train , y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [ ]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.
* Print the classification report.

In [ ]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)


In [ ]:
# Print the classification report for the model
from sklearn.metrics import classification_report
target_names = ["healthy loan", "high-risk loan)"]
print(classification_report(y_test, predictions, target_names=target_names))

---

## Create a Neural Network Model

### Step 1: Initialize a StandardScaler instance

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Step 2: Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5


nm = tf.keras.models.Sequential()

# First hidden layer
nm.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, activation="relu", input_dim = number_input_features))

# Second hidden layer
nm.add(tf.keras.layers.Dense(units = hidden_nodes_layer2 , activation = "sigmoid" ))

# Output layer
nm.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nm.summary()

In [ ]:
# Compile the model
nm.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nm.fit(X_train_scaled , y_train , epochs = 300)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nm.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Step 3: Optimize the model with the keras_tuner

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nm_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nm_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nm_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nm_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nm_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nm_model

In [ ]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [ ]:
 # Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
best_model.save("Project4.h5")